In [ ]:
import time

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv1D, Dense,  MaxPooling2D, MaxPooling1D, Flatten, Concatenate, GlobalAveragePooling2D, GlobalAveragePooling1D

In [ ]:
PATH = "/tmp/lhcf-cnn"
EPOCH = 2

In [ ]:
# Definisci lo schema del TFRecord
feature_description = {
    "posdE_01xy": tf.io.FixedLenFeature([384 * 384 * 2], tf.float32),
    "posdE_23x": tf.io.FixedLenFeature([384 * 2], tf.float32),
    "posdE_23y": tf.io.FixedLenFeature([384 * 2], tf.float32),
    "dE": tf.io.FixedLenFeature([16], tf.float32),
    "label": tf.io.FixedLenFeature([], tf.int64)
}

# Funzione per il parsing dei record
def parse_tfrecord_fn(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Ricostruisci le forme originali
    posdE_01xy = tf.reshape(example["posdE_01xy"], (384, 384, 2))
    posdE_23x = tf.reshape(example["posdE_23x"], (384, 2))
    posdE_23y = tf.reshape(example["posdE_23y"], (384, 2))
    dE = tf.reshape(example["dE"], (16,))
    label = example["label"]
    
    return {"posdE_01xy_input": posdE_01xy, "posdE_23x_input": posdE_23x, "posdE_23y_input": posdE_23y, "dE_input": dE}, label

# Carica e pre-processa i dati in batch
def load_dataset(tfrecord_file, batch_size=32, shuffle_buffer=1000):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord_fn, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle_buffer==None:
        dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    else:
        dataset = dataset.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Creazione dei dataset di training e validazione
train_dataset = load_dataset(f"{PATH}/train.tfrecord", batch_size=32)
validation_dataset = load_dataset(f"{PATH}/validation.tfrecord", batch_size=32, shuffle_buffer=None)

In [ ]:
train_dataset.element_spec

In [ ]:
train_dataset_noshuffle = load_dataset(f"{PATH}/train.tfrecord", batch_size=32, shuffle_buffer=None)

In [ ]:
# Conta il numero di esempi con label 0 e 1 nel train_dataset
count_label_0 = 0
count_label_1 = 0

# Itera su train_dataset per contare le label
for _, labels in train_dataset_noshuffle:
    # Converte i tensor delle label in numpy per operazioni semplici
    labels_numpy = labels.numpy()
    count_label_0 += np.sum(labels_numpy == 0)
    count_label_1 += np.sum(labels_numpy == 1)

print(f"Numero di esempi con label 0: {count_label_0}")
print(f"Numero di esempi con label 1: {count_label_1}")
ratio = count_label_0/ count_label_1
print("Rapporto: ",ratio)

In [ ]:
# Estrai un batch dal dataset
for batch in train_dataset_noshuffle.take(1):  # Prende il primo batch
    example = batch[0]  # Prende le feature
    label = batch[1]  # Prende la label
    break

# Estrai un singolo esempio
posdE_01xy_example = example["posdE_01xy_input"].numpy()[2]  # Prende il primo esempio del batch

# Visualizza l'immagine posdE_01xy_input
plt.imshow(posdE_01xy_example[:, :, 0], cmap='viridis')  # Visualizza il primo canale
plt.colorbar()
plt.title("posdE_01xy_input - Piano 1")
plt.show()

# Per visualizzare il secondo canale separatamente
plt.imshow(posdE_01xy_example[:, :, 1], cmap='viridis')  # Visualizza il secondo canale
plt.colorbar()
plt.title("posdE_01xy_input - Piano 2")
plt.show()

In [ ]:
# Estrai un batch dal dataset
for batch in train_dataset_noshuffle.take(1):  # Prende il primo batch
    example = batch[0]  # Prende le feature
    label = batch[1]  # Prende la label
    break

# Estrai un singolo esempio
posdE_23x_example = example["posdE_23x_input"].numpy()[2]  # Prende il primo esempio del batch

# Visualizza i due canali come linee separate
plt.figure(figsize=(10, 6))
plt.plot(posdE_23x_example[:, 0], label="Piano 2")
plt.plot(posdE_23x_example[:, 1], label="Piano 3")
plt.title("posdE_23x_input")
plt.xlabel("Indice")
plt.ylabel("Valore")
plt.legend()
plt.show()

In [ ]:
# Estrai un batch dal dataset
for batch in train_dataset_noshuffle.take(1):  # Prende il primo batch
    example = batch[0]  # Prende le feature
    label = batch[1]  # Prende la label
    break

# Estrai un singolo esempio
posdE_23y_example = example["posdE_23y_input"].numpy()[2]  # Prende il primo esempio del batch

# Visualizza i due canali come linee separate
plt.figure(figsize=(10, 6))
plt.plot(posdE_23y_example[:, 0], label="Piano 2")
plt.plot(posdE_23y_example[:, 1], label="Piano 3")
plt.title("posdE_23y_input")
plt.xlabel("Indice")
plt.ylabel("Valore")
plt.legend()
plt.show()

In [ ]:
# Estrai un batch dal dataset
for batch in train_dataset_noshuffle.take(1):  # Prende il primo batch
    example = batch[0]  # Prende le feature
    label = batch[1]  # Prende la label
    break

# Estrai un singolo esempio di dE_input
dE_example = example["dE_input"].numpy()[2]  # Prende il primo esempio del batch

# Visualizza dE_input come grafico a barre
plt.figure(figsize=(8, 5))
plt.bar(range(len(dE_example)), dE_example)
plt.title("dE_input")
plt.xlabel("Indice")
plt.ylabel("Valore")
plt.show()

In [ ]:
# Definizione della rete neurale

# Input per Conv2D con GlobalAveragePooling
input_posdE_01xy = Input(shape=(384, 384, 2), name="posdE_01xy_input")
x1 = Conv2D(4, (3, 3), activation="relu", padding="same")(input_posdE_01xy)
x1 = MaxPooling2D((2, 2))(x1)
#x1 = GlobalAveragePooling2D()(x1)  # Riducono i parametri
x1 = Flatten()(x1) 

# Input per Conv1D con GlobalAveragePooling
input_posdE_23x = Input(shape=(384, 2), name="posdE_23x_input")
x2 = Conv1D(4, 3, activation="relu", padding="same")(input_posdE_23x)
x2 = MaxPooling1D(2)(x2)
#x2 = GlobalAveragePooling1D()(x2)
x2 = Flatten()(x2)

input_posdE_23y = Input(shape=(384, 2), name="posdE_23y_input")
x3 = Conv1D(4, 3, activation="relu", padding="same")(input_posdE_23y)
x3 = MaxPooling1D(2)(x3)
#x3 = GlobalAveragePooling1D()(x3)
x3 = Flatten()(x3)

# Input per Dense
input_dE = Input(shape=(16,), name="dE_input")
x4 = Dense(4, activation="relu")(input_dE)

# Concatenazione delle rappresentazioni dei quattro blocchi
x = Concatenate()([x1, x2, x3, x4])

# Output per classificazione binaria
output = Dense(1, activation="sigmoid", name="output")(x)

# Definisci il modello
model = Model(inputs=[input_posdE_01xy, input_posdE_23x, input_posdE_23y, input_dE], outputs=output)

In [ ]:
# Compila il modello
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Visualizza il sommario del modello
model.summary()

In [ ]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
# Definisci i pesi per le classi
#class_weight = {0: 1, 1: 3}  

# Allena il modello
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCH,
    #class_weight=class_weight
)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='Training Accuracy')
plt.plot(history.history['val_loss'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Calcola le etichette e le predizioni per il dataset di training senza shuffle
y_train_true = np.concatenate([y for _, y in train_dataset_noshuffle.as_numpy_iterator()], axis=0)
train_predictions = model.predict(train_dataset_noshuffle)

# Dividi le predizioni del training in base alle etichette
train_predictions_0 = train_predictions[y_train_true == 0]
train_predictions_1 = train_predictions[y_train_true == 1]

# Calcola le etichette e le predizioni per il dataset di validazione senza shuffle
y_val_true = np.concatenate([y for _, y in validation_dataset.as_numpy_iterator()], axis=0)
val_predictions = model.predict(validation_dataset)

# Dividi le predizioni della validazione in base alle etichette
val_predictions_0 = val_predictions[y_val_true == 0]
val_predictions_1 = val_predictions[y_val_true == 1]


In [ ]:
# Plottaggio degli istogrammi normalizzati
plt.figure(figsize=(10, 6))

# Istogramma per train_predictions_label_0 e train_predictions_label_1
# Normalizza ciascun istogramma affinché l'area totale sia 1
train_hist_0, bins_0, _ = plt.hist(train_predictions_0, bins=100, alpha=0.4, color='darkorange', label='Train - Label 0', edgecolor='black', density=True)
train_hist_1, bins_1, _ = plt.hist(train_predictions_1, bins=100, alpha=0.4, color='blue', label='Train - Label 1', edgecolor='black', density=True)

# Istogrammi per la validazione (senza visualizzazione)
val_hist_0, bin_val_0 = np.histogram(val_predictions_0, bins=100, density=True)
val_hist_1, bin_val_1 = np.histogram(val_predictions_1, bins=100, density=True)

plt.plot(bin_val_0[1:], val_hist_0, '*', color='darkorange', label='Validation - Label 0')
plt.plot(bin_val_1[1:], val_hist_1, '*', color='blue', label='Validation - Label 1')

# Aggiungi etichette e legenda
plt.xlabel("Valori delle Predizioni")
plt.ylabel("Densità Normalizzata")
plt.title("Istogramma Normalizzato delle Predizioni per Train e Validation")
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Predizione per l'intero dataset 
#y_train_true = np.concatenate([y for _, y in train_dataset_noshuffle.as_numpy_iterator()], axis=0)
#train_predictions = model.predict(train_dataset_noshuffle)
y_train_pred = (train_predictions >= 0.5).astype(int)

#y_val_true = np.concatenate([y for _, y in validation_dataset.as_numpy_iterator()], axis=0)
#val_predictions = model.predict(validation_dataset)
y_val_pred = (val_predictions >= 0.5).astype(int)

# Calcola la matrice di confusione per il training
train_cm = confusion_matrix(y_train_true, y_train_pred)
ConfusionMatrixDisplay(train_cm, display_labels=['Label 0', 'Label 1']).plot()
plt.title('Confusion Matrix - Training')
plt.show()

# Calcola la matrice di confusione per la validazione
val_cm = confusion_matrix(y_val_true, y_val_pred)
ConfusionMatrixDisplay(val_cm, display_labels=['Label 0', 'Label 1']).plot()
plt.title('Confusion Matrix - Validation')
plt.show()